# GP-GAN implementation with Auto-masking enabled

## Pre-Processing input data

Download source and destination videos

In [ ]:
!gdown https://drive.google.com/uc?id=18tWHN1r-wEAEbtauCXSDcQUl3amf8MFK
!gdown https://drive.google.com/uc?id=1dM7NL68r01oXLmOaYu9WngZvjyf1Nxh3

Downloading...
From: https://drive.google.com/uc?id=18tWHN1r-wEAEbtauCXSDcQUl3amf8MFK
To: /content/src.mp4
72.3MB [00:00, 174MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1dM7NL68r01oXLmOaYu9WngZvjyf1Nxh3
To: /content/dst.mp4
122MB [00:00, 125MB/s]


Extract frames from uploaded video <br>
For **both** source and destination videos and store in respective folders

In [ ]:
import cv2, os, time

print("Current working direcotry: ", os.getcwd())
filename = input("Enter filename of video to be extracted (with extension): ")
foldername = input("Enter folder name where frames are to be stored: ").strip()
try:
	os.mkdir(foldername)
except:
	pass
frame_name = foldername+'frames'
fps = input("Enter video's frame per second: ")
t = input("Enter duration of video to be extracted (0 if full duration) (in seconds): ")
tot_frames = int(int(t) * int(fps))
start = time.time()
path= os.getcwd()+'/'+filename
cap = cv2.VideoCapture(path)
i=0
while(cap.isOpened()):
    if i> tot_frames and tot_frames != 0:
        break
    ret, frame_rgba = cap.read()
    if ret == False:
        break
    frame_rgb = cv2.cvtColor(frame_rgba, cv2.COLOR_RGBA2RGB)
    cv2.imwrite(foldername+'/'+frame_name+str(i)+'.jpg',frame_rgb)
    i=i+1
 
cap.release()
cv2.destroyAllWindows()
end = time.time()
print (f"Finished task in {end -  start:.2f} seconds")

Current working direcotry:  /content
Enter filename of video to be extracted (with extension): dst.mp4
Enter folder name where frames are to be stored: dst
Enter video's frame per second: 30
Enter duration of video to be extracted (0 if full duration) (in seconds): 5
Finished task in 3.09 seconds


Make 2 folders named rst, mask

In [ ]:
mkdir {rst,mask}

## Semantic Segmentation

### Download and install MIT CSAIL Semantic Segmentation repo

In [ ]:
!git clone https://github.com/CSAILVision/semantic-segmentation-pytorch.git

Cloning into 'semantic-segmentation-pytorch'...
remote: Enumerating objects: 1170, done.
remote: Total 1170 (delta 0), reused 0 (delta 0), pack-reused 1170
Receiving objects: 100% (1170/1170), 5.04 MiB | 27.16 MiB/s, done.
Resolving deltas: 100% (707/707), done.


In [ ]:
%cd semantic-segmentation-pytorch/

/content/semantic-segmentation-pytorch


Install required libs

In [ ]:
!pip install yacs

Download Pre-trained model and run trial on test image

In [ ]:
!bash demo_test.sh

--2021-06-25 14:27:21--  http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth
Resolving sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)... 128.30.195.26
Connecting to sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)|128.30.195.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95015426 (91M)
Saving to: ‘ckpt/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth’

encoder_epoch_20.pt 100%[===================>]  90.61M  31.8MB/s    in 2.8s    

2021-06-25 14:27:24 (31.8 MB/s) - ‘ckpt/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth’ saved [95015426/95015426]

--2021-06-25 14:27:24--  http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/decoder_epoch_20.pth
Resolving sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)... 128.30.195.26
Connecting to sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)|128.30.195.26|:80... connected.
HTTP request sent, awa

### Implementing Semantic Segmentation and generating masks

***Change these files according to try10*** 

1.   /content/semantic-segmentation-pytorch/config/   **ade20k-resnet50dilated-ppm_deepsup.yaml**
2.   /content/semantic-segmentation-pytorch/ **test.py**

*That will change how the segmented images will save and where they will save*

In [ ]:
!rm /content/semantic-segmentation-pytorch/config/ade20k-resnet50dilated-ppm_deepsup.yaml
!rm /content/semantic-segmentation-pytorch/test.py
%cd config
!gdown https://drive.google.com/uc?id=1QUsOynkHENZtMbQJMA5zqToKct5f1yac
%cd ..
!gdown https://drive.google.com/uc?id=1SXrCLmH9_bwl3dQlTHMkJuXLj02vqfai

/content/semantic-segmentation-pytorch/config
Downloading...
From: https://drive.google.com/uc?id=1QUsOynkHENZtMbQJMA5zqToKct5f1yac
To: /content/semantic-segmentation-pytorch/config/ade20k-resnet50dilated-ppm_deepsup.yaml
100% 765/765 [00:00<00:00, 2.98MB/s]
/content/semantic-segmentation-pytorch
Downloading...
From: https://drive.google.com/uc?id=1SXrCLmH9_bwl3dQlTHMkJuXLj02vqfai
To: /content/semantic-segmentation-pytorch/test.py
100% 5.90k/5.90k [00:00<00:00, 5.50MB/s]


Semantically segment all images in the source folder, to generate masks later

In [ ]:
%mkdir result
!python3 -u test.py --imgs /content/src

[2021-06-25 14:28:04,578 INFO test.py line 172 203] Loaded configuration file config/ade20k-resnet50dilated-ppm_deepsup.yaml
[2021-06-25 14:28:04,578 INFO test.py line 173 203] Running with config:
DATASET:
  imgMaxSize: 1000
  imgSizes: (300, 375, 450, 525, 600)
  list_train: ./data/training.odgt
  list_val: ./data/validation.odgt
  num_class: 150
  padding_constant: 8
  random_flip: True
  root_dataset: ./data/
  segm_downsampling_rate: 8
DIR: ckpt/ade20k-resnet50dilated-ppm_deepsup
MODEL:
  arch_decoder: ppm_deepsup
  arch_encoder: resnet50dilated
  fc_dim: 2048
  weights_decoder: 
  weights_encoder: 
TEST:
  batch_size: 1
  checkpoint: epoch_20.pth
  result: ./result
TRAIN:
  batch_size_per_gpu: 2
  beta1: 0.9
  deep_sup_scale: 0.4
  disp_iter: 20
  epoch_iters: 5000
  fix_bn: False
  lr_decoder: 0.02
  lr_encoder: 0.02
  lr_pow: 0.9
  num_epoch: 20
  optim: SGD
  seed: 304
  start_epoch: 0
  weight_decay: 0.0001
  workers: 16
VAL:
  batch_size: 1
  checkpoint: epoch_20.pth
  visua

Convert segmented images into masks [Eg. 140140140 for road,pavement] 

In [ ]:
import cv2, os
from os.path import isfile, join
import numpy as np

rgb = input("Enter rgb value of thing to copy (with leading zeros): ")
rgblist=[]
a=6
b=9
for i in range (3):
  rgblist.append(int(rgb[a:b]))
  a=a-3
  b=b-3
print(rgblist)
#Path of folder where source images are located
pathIn = "/content/semantic-segmentation-pytorch/result/" 
#Path to save results
rst = "/content/mask/"

files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
for i in range(len(files)):
	filename=pathIn + files[i]
	image = cv2.imread(filename)
	image[ np.where(( image != rgblist).all( axis=2 ))] = [0,0,0]
	image[ np.where(( image == rgblist).all( axis=2  ))] = [1,1,1]
	cv2.imwrite(rst+"mask-"+files[i],image)
	print(rst+"mask-"+files[i]+"\t...saved")


Enter rgb value of thing to copy (with leading zeros): 140140140
[140, 140, 140]
/content/mask/mask-srcframes90.png	...saved
/content/mask/mask-srcframes133.png	...saved
/content/mask/mask-srcframes85.png	...saved
/content/mask/mask-srcframes123.png	...saved
/content/mask/mask-srcframes80.png	...saved
/content/mask/mask-srcframes96.png	...saved
/content/mask/mask-srcframes39.png	...saved
/content/mask/mask-srcframes89.png	...saved
/content/mask/mask-srcframes16.png	...saved
/content/mask/mask-srcframes35.png	...saved
/content/mask/mask-srcframes86.png	...saved
/content/mask/mask-srcframes147.png	...saved
/content/mask/mask-srcframes31.png	...saved
/content/mask/mask-srcframes137.png	...saved
/content/mask/mask-srcframes144.png	...saved
/content/mask/mask-srcframes69.png	...saved
/content/mask/mask-srcframes9.png	...saved
/content/mask/mask-srcframes117.png	...saved
/content/mask/mask-srcframes20.png	...saved
/content/mask/mask-srcframes49.png	...saved
/content/mask/mask-srcframes44.png

In [ ]:
cd ..

/content


## GP GAN Implementation

Download GP GAN repo

In [ ]:
!git clone https://github.com/wuhuikai/GP-GAN.git

Cloning into 'GP-GAN'...
remote: Enumerating objects: 756, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 756 (delta 50), reused 72 (delta 32), pack-reused 650
Receiving objects: 100% (756/756), 18.46 MiB | 16.25 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [ ]:
%cd GP-GAN

/content/GP-GAN


In [ ]:
%cd models

/content/GP-GAN/models


Downloading the pretrained models

In [ ]:
!gdown https://drive.google.com/uc?id=0Bybnpq8dvwudXzZFLU1iOXVOZ3c
!gdown https://drive.google.com/uc?id=0Bybnpq8dvwudRzRLUm1xYmt6R1E
!gdown https://drive.google.com/uc?id=0Bybnpq8dvwudOEFELWkzcGJJVjA

Downloading...
From: https://drive.google.com/uc?id=0Bybnpq8dvwudXzZFLU1iOXVOZ3c
To: /content/GP-GAN/models/unsupervised_blending_gan.npz
13.3MB [00:00, 62.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=0Bybnpq8dvwudRzRLUm1xYmt6R1E
To: /content/GP-GAN/models/realismCNN_all_iter3.npz
499MB [00:04, 118MB/s]
Downloading...
From: https://drive.google.com/uc?id=0Bybnpq8dvwudOEFELWkzcGJJVjA
To: /content/GP-GAN/models/blending_gan.npz
264MB [00:03, 86.0MB/s]


In [ ]:
%cd ..

/content/GP-GAN


Installing required libraries including chainer, cupy and scikit-image

In [ ]:
pip install -r /content/GP-GAN/requirements/test/requirements.txt

     |████████████████████████████████| 880kB 7.6MB/s 
     |████████████████████████████████| 3.1MB 20.8MB/s 
     |████████████████████████████████| 26.3MB 108kB/s 
     |████████████████████████████████| 1.2MB 45.9MB/s 
  Created wheel for chainer: filename=chainer-6.3.0-cp37-none-any.whl size=883600 sha256=3ad2132a2471de1ec77a57ede32e65be095cfee450eca9d0faf665d78c7fb48e
  Stored in directory: /root/.cache/pip/wheels/bf/fb/7d/cda50b99114b987a82c3819bebee7259e20e5a1d6113b3f6ac
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
Successfully built chainer
Failed to build cupy
ERROR: tensorflow 2.5.0 has requirement protobuf>=3.9.2, but you'll have protobuf 3.7.1 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement typing-extensions~=3.7.4, but you'll have typing-extensions 3.6.6 which is incompatible.
ERROR: tensorflow-hub 0.12.0 has requirement protobuf>=3.8.0, but you'll have protobuf 3.7.1 which is incompatible.
ERROR: pymc3 3.11.2 has requirement ty

Using subprocess to call the function and pass values apropriately, to run Blending process

In [ ]:
import os, cv2 ,natsort, subprocess
src_add = "/content/src/"
dst_add = "/content/dst/"
mask_add = "/content/mask/"
rst_add = "/content/rst/"

def frame_blend(src_add, dst_add, mask_add, rst_add):
	src = natsort.natsorted(os.listdir(src_add))
	dst = natsort.natsorted(os.listdir(dst_add))
	mask = natsort.natsorted(os.listdir(mask_add))
	arg = ['python', 'run_gp_gan.py', '--src_image', 'images/test_images/src.jpg', '--dst_image', 'images/test_images/dst.jpg', '--mask_image', 'images/test_images/mask.png', '--blended_image', 'images/test_images/result_image.png'] 
	if len(src)>len(dst):
		l = len(dst)
	else:
		l = len(src)
	for i in range(l):
		print (src[i] , end="\t")
		print (dst[i])
		arg[3] = src_add + src[i]
		arg[5] = dst_add + dst[i]
		arg[7] = mask_add + mask[i]
		arg[-1] = rst_add + "result-"+str(i)+".png"
		#print(arg)
		p=subprocess.run(arg,capture_output=True)
		#print(p.stdout)
	
		
		
frame_blend(src_add, dst_add, mask_add, rst_add)

srcframes0.jpg	dstframes0.jpg
srcframes1.jpg	dstframes1.jpg
srcframes2.jpg	dstframes2.jpg
srcframes3.jpg	dstframes3.jpg
srcframes4.jpg	dstframes4.jpg
srcframes5.jpg	dstframes5.jpg
srcframes6.jpg	dstframes6.jpg
srcframes7.jpg	dstframes7.jpg
srcframes8.jpg	dstframes8.jpg
srcframes9.jpg	dstframes9.jpg
srcframes10.jpg	dstframes10.jpg
srcframes11.jpg	dstframes11.jpg
srcframes12.jpg	dstframes12.jpg
srcframes13.jpg	dstframes13.jpg
srcframes14.jpg	dstframes14.jpg
srcframes15.jpg	dstframes15.jpg
srcframes16.jpg	dstframes16.jpg
srcframes17.jpg	dstframes17.jpg
srcframes18.jpg	dstframes18.jpg
srcframes19.jpg	dstframes19.jpg
srcframes20.jpg	dstframes20.jpg
srcframes21.jpg	dstframes21.jpg
srcframes22.jpg	dstframes22.jpg
srcframes23.jpg	dstframes23.jpg
srcframes24.jpg	dstframes24.jpg
srcframes25.jpg	dstframes25.jpg
srcframes26.jpg	dstframes26.jpg
srcframes27.jpg	dstframes27.jpg
srcframes28.jpg	dstframes28.jpg
srcframes29.jpg	dstframes29.jpg
srcframes30.jpg	dstframes30.jpg
srcframes31.jpg	dstframes31.

In [ ]:
cd ..

/content


## Post-Processing output data

### Combining resulting frames into a .mp4 video at 30 fps

In [ ]:
import cv2 ,natsort
import numpy as np
import os

from os.path import isfile, join

def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    
    #for sorting the file names properly
    files = natsort.natsorted(files)

    for i in range(len(files)):
        filename=pathIn + files[i]
        print("######################", filename)
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(pathOut, fourcc, fps, size)

    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

def main():
    #Folder containing frames to combine
    pathIn= 'rst/'
    pathOut = 'Blended.mp4'
    fps = 30.0
    convert_frames_to_video(pathIn, pathOut, fps)

if __name__=="__main__":
    main()

###################### rst/result-0.png
rst/result-0.png
###################### rst/result-1.png
rst/result-1.png
###################### rst/result-2.png
rst/result-2.png
###################### rst/result-3.png
rst/result-3.png
###################### rst/result-4.png
rst/result-4.png
###################### rst/result-5.png
rst/result-5.png
###################### rst/result-6.png
rst/result-6.png
###################### rst/result-7.png
rst/result-7.png
###################### rst/result-8.png
rst/result-8.png
###################### rst/result-9.png
rst/result-9.png
###################### rst/result-10.png
rst/result-10.png
###################### rst/result-11.png
rst/result-11.png
###################### rst/result-12.png
rst/result-12.png
###################### rst/result-13.png
rst/result-13.png
###################### rst/result-14.png
rst/result-14.png
###################### rst/result-15.png
rst/result-15.png
###################### rst/result-16.png
rst/result-16.png
#################